In [ ]:
import base
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

sns.set_style('ticks')
sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

### Setup

- Load data
- Add metadata
- Draw gates
- Gate transfected cells

Result from this section: DataFrame `df` representing transfected cells.

Load all transfection data collected as of 2024.04.11

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'/'kasey'

exp90_path = base_path/'2024.03.31_exp90'/'export'
exp90_2_path = base_path/'2024.04.02_exp90.2'/'export'
exp90_3_path = base_path/'2024.04.02_exp90.3'/'export'
exp90_4_path = base_path/'2024.04.05_exp90.4'/'export'
exp91_path = base_path/'2024.04.08_exp91'/'export'
exp92_path = base_path/'2024.04.12_exp92'/'export'

plates = pd.DataFrame({
    'data_path': [exp90_path/'plate1', exp90_path/'plate2', 
                  exp90_2_path, exp90_4_path,
                  exp90_3_path/'plate1', exp90_3_path/'plate2', 
                  exp91_path/'plate1.1', exp91_path/'plate1.2', exp91_path/'plate1.3', 
                  exp91_path/'plate2.1', exp91_path/'plate2.2', exp91_path/'plate2.3',
                  exp92_path/'plate1.1', exp92_path/'plate1.2', exp92_path/'plate1.3', 
                  exp92_path/'plate2.1', exp92_path/'plate2.2', exp92_path/'plate2.3',],
    
    'yaml_path': ([exp90_path/'exp90_plate1_wells.yaml', exp90_path/'exp90_plate2_wells.yaml', 
                   exp90_path/'exp90_plate2_wells.yaml', exp90_path/'exp90_plate1_wells.yaml',
                   exp90_path/'exp90_plate1_wells.yaml', exp90_path/'exp90_plate2_wells.yaml', ] +
                  [exp91_path/'exp91_plate1_wells.yaml']*3 + 
                  [exp91_path/'exp91_plate2.1_wells.yaml', exp91_path/'exp91_plate2.2_wells.yaml', exp91_path/'exp91_plate2.3_wells.yaml'] +
                  [exp92_path/'exp92_plate1_wells.yaml', exp92_path/'exp92_plate1.2_wells.yaml', exp92_path/'exp92_plate1_wells.yaml',
                   exp92_path/'exp92_plate2_wells.yaml', exp92_path/'exp92_plate2.2_wells.yaml', exp92_path/'exp92_plate2_wells.yaml',]
                  ),
    
    'biorep': ([1, 1, 
                2, 2, 
                3, 3,] + 
                [1, 2, 3,]*4),
    
    'exp': (['exp90', 'exp90', 
             'exp90.2', 'exp90.4', 
             'exp90.3', 'exp90.3',] + 
            ['exp91']*6 + 
            ['exp92']*6)
})

output_path = rd.rootdir/'output'/'tuning'/'more'
cache_path = rd.rootdir/'output'/'tuning'/'data.gzip'

metadata_keys = set()
for p in plates['yaml_path'].unique():
    rd.plot.plot_well_metadata(p)
    metadata_keys.update(rd.flow.load_well_metadata(p).keys())
display(metadata_keys)

In [ ]:
# Load data
data = pd.DataFrame()
if cache_path.is_file(): data = pd.read_parquet(cache_path)
else: 
    channel_list = ['mCherry-A','mRuby2-A','FSC-A','SSC-A','tagBFP-A','mGL-A']
    data = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data = data[data[c]>0]
    
    data.dropna(inplace=True)
    data.to_parquet(rd.outfile(cache_path))
display(data)

In [ ]:
# Add metadata for constructs
metadata = base.get_metadata(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx')
data = data.merge(metadata, how='left', on='construct')
display(data)

# Create dicts to specify colors/markers
metadata['miR_'] = (metadata['miR'].isin(['none','na'])).map({True: '–', False:'+'})
metadata['ts_'] = (metadata['ts'].isin(['none','na'])).map({True: '–', False:'+'})
metadata_dict = metadata.set_index('construct').to_dict('dict')
construct_palette = metadata_dict['color']
construct_markers = metadata_dict['markers']

Draw gates based on untransfected population, then gate transfected cells

In [ ]:
gates = pd.DataFrame()
channel_list = ['tagBFP-A', 'mGL-A', 'mCherry-A', 'mRuby2-A']
for channel in channel_list:
    gates[channel] = data[data['construct']=='UT'].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates.reset_index(inplace=True)

# Add missing gates
gates.loc[len(gates.index)] = ['exp90.4',0,0,0,0,]  
gates.loc[gates['exp']=='exp90.4', channel_list] = gates.loc[gates['exp']=='exp90.2', channel_list].values

# Indicate which channels are relevant for each experiment
gates.sort_values(['exp'], inplace=True)
gates['marker'] = 'mGL-A'
gates['output'] = 'mRuby2-A'

display(gates)

Gate data per experiment based on transfection marker expression

In [ ]:
data = data.groupby('exp')[data.columns].apply(lambda x: base.gate_data(x,gates))
data.reset_index(inplace=True, drop=True)
max = 1e6
df = data[(data['expressing']) & (data['output']<max)]
display(df)

In [ ]:
# Bin by marker quantiles
by = ['construct','exp','biorep']
df['bin_marker_quantiles'] = df.groupby(by)['marker'].transform(lambda x: pd.qcut(x, q=20, duplicates='drop'))
quantiles = df.groupby(by+['bin_marker_quantiles'])['marker'].median().rename('bin_marker_quantiles_median').reset_index()
df_quantiles = df.merge(quantiles, how='left', on=by+['bin_marker_quantiles'])
display(df_quantiles)

In [ ]:
# Population stats
stat_list = [np.std, sp.stats.gmean, sp.stats.gstd, sp.stats.variation]
grouped = df_quantiles.groupby(by=['construct','biorep','exp'])
stats = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats.columns = stats.columns.map(lambda i: base.rename_multilevel_cols(i))
stats['count'] = grouped['output'].count().reset_index()['output']
stats = stats.merge(metadata, how='left', on='construct')
display(stats)

In [ ]:
# Quantile stats & slope
df_quantiles['bin_marker_quantiles_median_log'] = df_quantiles['bin_marker_quantiles_median'].apply(np.log10)
df_quantiles['output_log'] = df_quantiles['output'].apply(np.log10)

stat_list = [sp.stats.gmean, ]#sp.stats.gstd, sp.stats.variation]
grouped = df_quantiles.groupby(by=['construct','biorep','exp','bin_marker_quantiles_median'])
stats_quantiles = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats_quantiles.columns = stats_quantiles.columns.map(lambda i: base.rename_multilevel_cols(i))
stats_quantiles['count'] = grouped['output'].count().reset_index()['output']
stats_quantiles = stats_quantiles.merge(metadata, how='left', on='construct')
display(stats_quantiles)

stats_quantiles['bin_marker_quantiles_median_log'] = stats_quantiles['bin_marker_quantiles_median'].apply(np.log10)
stats_quantiles['output_gmean_log'] = stats_quantiles['output_gmean'].apply(np.log10)

In [ ]:
# Compute slope for all constructs
def get_slope(df):
    slope, intercept, r_value, p_value, stderr = sp.stats.linregress(df['bin_marker_quantiles_median_log'], df['output_gmean_log'])
    result = pd.DataFrame(columns=['slope', 'intercept_log', 'r_value', 'p_value', 'stderr'])
    result.loc[len(result.index)] = [slope, intercept, r_value, p_value, stderr]
    return result

fits = stats_quantiles.groupby(['construct','exp','biorep'])[stats_quantiles.columns].apply(get_slope).reset_index()
fits['intercept'] = fits['intercept_log'].apply(lambda x: 10**x)
fits = fits.merge(metadata, how='left', on='construct')
display(fits)

In [ ]:
# EF1a miR-FF5 design 1
fig, axes = plt.subplots(1,5, figsize=(14,5), gridspec_kw=dict(width_ratios=(1,0.3,0.3,0.3,0.3), wspace=0.4))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
plot_df = df_quantiles[(((df_quantiles['miR']=='miR.FF5') & (df_quantiles['group']=='controller') & (df_quantiles['design']==1) & (df_quantiles['ts']!='FF3x1')) | (df_quantiles['group']=='base')) &
                         (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==2)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=construct_palette, 
             legend=False, dashes=False, style='construct', markers=construct_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==2), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=construct_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==2)], y='output', hue='construct', palette=construct_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density', ylim=ylim, ylabel='', yticklabels=['']*(len(ax.get_yticklabels())-1))


# stat gmean
ax = axes[2]
plot_df = stats[(((stats['miR']=='miR.FF5') & (stats['group']=='controller') & (stats['design']==1) & (stats['ts']!='FF3x1')) | (stats['group']=='base')) &
                         (stats['promoter']=='EF1a')]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_gmean', hue='construct', palette=construct_palette, 
                    legend=False, ax=ax, marker=construct_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric\nmean', xlim=(-0.5, 2.5), xlabel='', ylim=ylim, ylabel='', yscale='log',
       yticklabels=['']*(len(ax.get_yticklabels())-1), xticklabels=[metadata_dict['ts_kind'][c.get_text()] for c in ax.get_xticklabels()])
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[3]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_std', hue='construct', palette=construct_palette, 
                    legend=False, ax=ax, marker=construct_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard\ndeviation', xlim=(-0.5, 2.5), xlabel='', ylim=ylim, ylabel='', yscale='log',
       yticklabels=['']*(len(ax.get_yticklabels())-1), xticklabels=[metadata_dict['ts_kind'][c.get_text()] for c in ax.get_xticklabels()])
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[4]
plot_df = fits[(((fits['miR']=='miR.FF5') & (fits['group']=='controller') & (fits['design']==1) & (fits['ts']!='FF3x1')) | (fits['group']=='base')) &
                         (fits['promoter']=='EF1a')]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='slope', hue='construct', palette=construct_palette, 
                    legend=False, ax=ax, marker=construct_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
    
    x_range = np.linspace(np.log10(2.5e2), np.log10(1.2e4), 1000)
    plot_df2 = group[group['biorep']==2]
    axes[0].plot(10**(x_range), 10**(plot_df2['slope'].values[0] * x_range + plot_df2['intercept_log'].values[0]), color='black', zorder=0)

ax.set(title='Slope', xlim=(-0.5, 2.5), xlabel='', ylim=(0,1.2), ylabel='',
       xticklabels=[metadata_dict['ts_kind'][c.get_text()] for c in ax.get_xticklabels()])
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(output_path/(f'line-marg-hist-stat-slope_EF1a-miR.FF5.svg')), bbox_inches='tight')

In [ ]:
# EF1a miR-FF5 design 1 controls
fig, axes = plt.subplots(1,5, figsize=(16,5), gridspec_kw=dict(width_ratios=(1,0.3,0.4,0.4,0.4), wspace=0.4))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
plot_df = df_quantiles[(df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==2) &
                       (((df_quantiles['miR']=='miR.FF5') & (df_quantiles['group'].isin(['controller','miR'])) & (df_quantiles['design'].isin([0,1])) & (df_quantiles['ts'].isin(['FF6x1','none'])) & (df_quantiles['miR_loc'].isin(['CDS','na']))) | 
                        (df_quantiles['group']=='base') | ((df_quantiles['group']=='ts3') & (df_quantiles['ts'].isin(['FF6x1','FF5x1']))))]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=construct_palette, 
             legend=False, dashes=False, style='construct', markers=construct_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==2), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=construct_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==2)], y='output', hue='construct', palette=construct_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density', ylim=ylim, ylabel='', yticklabels=['']*(len(ax.get_yticklabels())-1))


# stat gmean
ax = axes[2]
plot_df = stats[(stats['promoter']=='EF1a') & 
                       (((stats['miR']=='miR.FF5') & (stats['group'].isin(['controller','miR'])) & (stats['design'].isin([0,1])) & (stats['ts'].isin(['FF6x1','none'])) & (stats['miR_loc'].isin(['CDS','na']))) | 
                        (stats['group']=='base') | ((stats['group']=='ts3') & (stats['ts'].isin(['FF6x1','FF5x1']))))]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_gmean', hue='construct', palette=construct_palette, 
                    legend=False, ax=ax, marker=construct_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric\nmean', xlim=(-0.5, 5.5), xlabel='', ylim=ylim, ylabel='', yscale='log',
       yticklabels=['']*(len(ax.get_yticklabels())-1),)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata, 'construct', ['miR_', 'ts_'], ax=ax, annotate=False)

# stat std
ax = axes[3]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_std', hue='construct', palette=construct_palette, 
                    legend=False, ax=ax, marker=construct_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard\ndeviation', xlim=(-0.5, 5.5), xlabel='', ylim=ylim, ylabel='', yscale='log',
       yticklabels=['']*(len(ax.get_yticklabels())-1),)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata, 'construct', ['miR_', 'ts_'], ax=ax, annotate=False)

# slope
ax = axes[4]
plot_df = fits[(fits['promoter']=='EF1a') &
                       (((fits['miR']=='miR.FF5') & (fits['group'].isin(['controller','miR'])) & (fits['design'].isin([0,1])) & (fits['ts'].isin(['FF6x1','none'])) & (fits['miR_loc'].isin(['CDS','na']))) | 
                        (fits['group']=='base') | ((fits['group']=='ts3') & (fits['ts'].isin(['FF6x1','FF5x1']))))]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='slope', hue='construct', palette=construct_palette, 
                    legend=False, ax=ax, marker=construct_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
    
    x_range = np.linspace(np.log10(2.5e2), np.log10(1.2e4), 1000)
    plot_df2 = group[group['biorep']==2]
    axes[0].plot(10**(x_range), 10**(plot_df2['slope'].values[0] * x_range + plot_df2['intercept_log'].values[0]), color='black', zorder=0)

ax.set(title='Slope', xlim=(-0.5, 5.5), xlabel='', ylim=(0,1.2), ylabel='',)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata, 'construct', ['miR_', 'ts_'], ax=ax, annotate=False)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist-stat-slope_EF1a-miR.FF5-controls.svg')), bbox_inches='tight')

In [ ]:
# Create new color palette for comparing miR / miRE
metadata2 = metadata.copy()
metadata2.loc[(metadata['miR']=='miRE.FF5') & (metadata['ts_kind']=='T') & (metadata['group']=='controller'), 'color'] = base.colors['orange']
metadata2.loc[(metadata['miR']=='miRE.FF4') & (metadata['ts_kind']=='T') & (metadata['group']=='controller'), 'color'] = base.colors['orange']
metadata2.loc[(metadata['miR']=='miRE.FF5') & (metadata['ts_kind']=='NT'), 'color'] = metadata2.loc[(metadata['miR']=='miRE.FF5') & (metadata['ts_kind']=='NT'), 'color'].apply(base.get_light_color)
metadata2.loc[(metadata['miR']=='miRE.FF4') & (metadata['ts_kind']=='NT'), 'color'] = metadata2.loc[(metadata['miR']=='miRE.FF4') & (metadata['ts_kind']=='NT'), 'color'].apply(base.get_light_color)
metadata2.loc[(metadata['miR']=='miRE.FF5') & (metadata['ts_kind']=='NT'), 'markers'] = 'P'
metadata2.loc[(metadata['miR']=='miRE.FF4') & (metadata['ts_kind']=='NT'), 'markers'] = 'P' #'v'
metadata2.loc[(metadata['miR']=='miRE.FF5') & (metadata['ts_kind']=='T'), 'markers'] = 'D'
metadata2.loc[(metadata['miR']=='miRE.FF4') & (metadata['ts_kind']=='T'), 'markers'] = 'D' #'^'
metadata_dict2 = metadata2.set_index('construct').to_dict('dict')
miRE_palette = metadata_dict2['color']
miRE_markers = metadata_dict2['markers']

In [ ]:
# EF1a miR-FF5 vs miRE-FF5 design 1
fig, axes = plt.subplots(1,5, figsize=(15,5), gridspec_kw=dict(width_ratios=(1,0.3,0.4,0.4,0.4), wspace=0.4))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
plot_df = df_quantiles[(((df_quantiles['miR'].isin(['miR.FF5','miRE.FF5'])) & (df_quantiles['group']=='controller') & (df_quantiles['design']==1) & (df_quantiles['ts']!='FF3x1')) | (df_quantiles['group']=='base')) &
                         (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==2)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=miRE_palette, 
             legend=False, dashes=False, style='construct', markers=miRE_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==2), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=miRE_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==2)], y='output', hue='construct', palette=construct_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density', ylim=ylim, ylabel='', yticklabels=['']*(len(ax.get_yticklabels())-1))


# stat gmean
ax = axes[2]
plot_df = stats[(((stats['miR'].isin(['miR.FF5','miRE.FF5'])) & (stats['group']=='controller') & (stats['design']==1) & (stats['ts']!='FF3x1')) | (stats['group']=='base')) &
                         (stats['promoter']=='EF1a')]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_gmean', hue='construct', palette=miRE_palette, 
                    legend=False, ax=ax, marker=miRE_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric\nmean', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',
       yticklabels=['']*(len(ax.get_yticklabels())-1), xticklabels=[metadata_dict['ts_kind'][c.get_text()] for c in ax.get_xticklabels()])
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[3]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_std', hue='construct', palette=miRE_palette, 
                    legend=False, ax=ax, marker=miRE_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard\ndeviation', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',
       yticklabels=['']*(len(ax.get_yticklabels())-1), xticklabels=[metadata_dict['ts_kind'][c.get_text()] for c in ax.get_xticklabels()])
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[4]
plot_df = fits[(((fits['miR'].isin(['miR.FF5','miRE.FF5'])) & (fits['group']=='controller') & (fits['design']==1) & (fits['ts']!='FF3x1')) | (fits['group']=='base')) &
                         (fits['promoter']=='EF1a')]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='slope', hue='construct', palette=miRE_palette, 
                    legend=False, ax=ax, marker=miRE_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
    
    x_range = np.linspace(np.log10(2.5e2), np.log10(1.2e4), 1000)
    plot_df2 = group[group['biorep']==2]
    axes[0].plot(10**(x_range), 10**(plot_df2['slope'].values[0] * x_range + plot_df2['intercept_log'].values[0]), color='black', zorder=0)

ax.set(title='Slope', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), xlabel='', ylim=(0,1.2), ylabel='',
       xticklabels=[metadata_dict['ts_kind'][c.get_text()] for c in ax.get_xticklabels()])
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(output_path/(f'line-marg-hist-stat-slope_EF1a-miRE.FF5.svg')), bbox_inches='tight')

In [ ]:
# Create new color palette for comparing FF4 to FF5
metadata2 = metadata.copy()
metadata2.loc[(metadata['miR']=='miRE.FF5') & (metadata['ts_kind']=='T') & (metadata['group']=='controller'), 'color'] = base.colors['orange']
metadata2.loc[(metadata['miR']=='miRE.FF4') & (metadata['ts_kind']=='T') & (metadata['group']=='controller'), 'color'] = base.colors['red']
metadata2.loc[(metadata['miR']=='miRE.FF5') & (metadata['ts_kind']=='NT'), 'color'] = metadata2.loc[(metadata['miR']=='miRE.FF5') & (metadata['ts_kind']=='NT'), 'color'].apply(base.get_light_color)
metadata2.loc[(metadata['miR']=='miRE.FF4') & (metadata['ts_kind']=='NT'), 'color'] = metadata2.loc[(metadata['miR']=='miRE.FF4') & (metadata['ts_kind']=='NT'), 'color']
metadata2.loc[(metadata['miR']=='miRE.FF5') & (metadata['ts_kind']=='NT'), 'markers'] = 'P'
metadata2.loc[(metadata['miR']=='miRE.FF4') & (metadata['ts_kind']=='NT'), 'markers'] = 'X'
metadata2.loc[(metadata['miR']=='miRE.FF5') & (metadata['ts_kind']=='T'), 'markers'] = 'D'
metadata2.loc[(metadata['miR']=='miRE.FF4') & (metadata['ts_kind']=='T'), 'markers'] = 'o'
metadata_dict2 = metadata2.set_index('construct').to_dict('dict')
FF_palette = metadata_dict2['color']
FF_markers = metadata_dict2['markers']

In [ ]:
# EF1a miRE-FF5 vs miRE-FF4 design 1
fig, axes = plt.subplots(1,5, figsize=(15,5), gridspec_kw=dict(width_ratios=(1,0.3,0.4,0.4,0.4), wspace=0.4))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
plot_df = df_quantiles[(((df_quantiles['miR'].isin(['miRE.FF5','miRE.FF4'])) & (df_quantiles['group']=='controller') & (df_quantiles['design']==1) & (df_quantiles['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (df_quantiles['group']=='base')) &
                         (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==2)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=FF_palette, 
             legend=False, dashes=False, style='construct', markers=FF_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==3), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=FF_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==2)], y='output', hue='construct', palette=construct_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density', ylim=ylim, ylabel='', yticklabels=['']*(len(ax.get_yticklabels())-1))


# stat gmean
ax = axes[2]
plot_df = stats[(((stats['miR'].isin(['miRE.FF5','miRE.FF4'])) & (stats['group']=='controller') & (stats['design']==1) & (stats['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (stats['group']=='base')) &
                         (stats['promoter']=='EF1a')]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_gmean', hue='construct', palette=FF_palette, 
                    legend=False, ax=ax, marker=FF_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric\nmean', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',
       yticklabels=['']*(len(ax.get_yticklabels())-1), xticklabels=[metadata_dict['ts_kind'][c.get_text()] for c in ax.get_xticklabels()])
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[3]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_std', hue='construct', palette=FF_palette, 
                    legend=False, ax=ax, marker=FF_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard\ndeviation', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',
       yticklabels=['']*(len(ax.get_yticklabels())-1), xticklabels=[metadata_dict['ts_kind'][c.get_text()] for c in ax.get_xticklabels()])
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[4]
plot_df = fits[(((fits['miR'].isin(['miRE.FF5','miRE.FF4'])) & (fits['group']=='controller') & (fits['design']==1) & (fits['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (fits['group']=='base')) &
                         (fits['promoter']=='EF1a')]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='slope', hue='construct', palette=FF_palette, 
                    legend=False, ax=ax, marker=FF_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
    
    x_range = np.linspace(np.log10(2.5e2), np.log10(1.2e4), 1000)
    plot_df2 = group[group['biorep']==2]
    axes[0].plot(10**(x_range), 10**(plot_df2['slope'].values[0] * x_range + plot_df2['intercept_log'].values[0]), color='black', zorder=0)

ax.set(title='Slope', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), xlabel='', ylim=(0,1.2), ylabel='',
       xticklabels=[metadata_dict['ts_kind'][c.get_text()] for c in ax.get_xticklabels()])
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(output_path/(f'line-marg-hist-stat-slope_EF1a-miRE.FF4.svg')), bbox_inches='tight')

In [ ]:
# EF1a miR-FF4 design 1 controls
fig, axes = plt.subplots(1,5, figsize=(16,5), gridspec_kw=dict(width_ratios=(1,0.3,0.4,0.4,0.4), wspace=0.4))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
plot_df = df_quantiles[(df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==2) &
                       (((df_quantiles['miR']=='miR.FF4') & (df_quantiles['group'].isin(['controller','miR'])) & (df_quantiles['design'].isin([0,1])) & (df_quantiles['ts'].isin(['FF6x1','none'])) & (df_quantiles['miR_loc'].isin(['CDS','na']))) | 
                        (df_quantiles['group']=='base') | ((df_quantiles['group']=='ts3') & (df_quantiles['ts'].isin(['FF6x1','FF4x1']))))]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=construct_palette, 
             legend=False, dashes=False, style='construct', markers=construct_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==2), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=construct_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==2)], y='output', hue='construct', palette=construct_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density', ylim=ylim, ylabel='', yticklabels=['']*(len(ax.get_yticklabels())-1))


# stat gmean
ax = axes[2]
plot_df = stats[(stats['promoter']=='EF1a') & 
                       (((stats['miR']=='miR.FF4') & (stats['group'].isin(['controller','miR'])) & (stats['design'].isin([0,1])) & (stats['ts'].isin(['FF6x1','none'])) & (stats['miR_loc'].isin(['CDS','na']))) | 
                        (stats['group']=='base') | ((stats['group']=='ts3') & (stats['ts'].isin(['FF6x1','FF4x1']))))]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_gmean', hue='construct', palette=construct_palette, 
                    legend=False, ax=ax, marker=construct_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric\nmean', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',
       yticklabels=['']*(len(ax.get_yticklabels())-1),)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata, 'construct', ['miR_', 'ts_'], ax=ax, annotate=False)

# stat std
ax = axes[3]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_std', hue='construct', palette=construct_palette, 
                    legend=False, ax=ax, marker=construct_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard\ndeviation', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',
       yticklabels=['']*(len(ax.get_yticklabels())-1),)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata, 'construct', ['miR_', 'ts_'], ax=ax, annotate=False)

# slope
ax = axes[4]
plot_df = fits[(fits['promoter']=='EF1a') &
                       (((fits['miR']=='miR.FF4') & (fits['group'].isin(['controller','miR'])) & (fits['design'].isin([0,1])) & (fits['ts'].isin(['FF6x1','none'])) & (fits['miR_loc'].isin(['CDS','na']))) | 
                        (fits['group']=='base') | ((fits['group']=='ts3') & (fits['ts'].isin(['FF6x1','FF4x1']))))]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='slope', hue='construct', palette=construct_palette, 
                    legend=False, ax=ax, marker=construct_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
    
    x_range = np.linspace(np.log10(2.5e2), np.log10(1.2e4), 1000)
    plot_df2 = group[group['biorep']==2]
    axes[0].plot(10**(x_range), 10**(plot_df2['slope'].values[0] * x_range + plot_df2['intercept_log'].values[0]), color='black', zorder=0)

ax.set(title='Slope', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), xlabel='', ylim=(0,1.2), ylabel='',)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata, 'construct', ['miR_', 'ts_'], ax=ax, annotate=False)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist-stat-slope_EF1a-miR.FF4-controls.svg')), bbox_inches='tight')

In [ ]:
# EF1a miRE-FF5 design 1 controls
fig, axes = plt.subplots(1,5, figsize=(16,5), gridspec_kw=dict(width_ratios=(1,0.3,0.4,0.4,0.4), wspace=0.4))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
plot_df = df_quantiles[(df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==2) &
                       (((df_quantiles['miR'].isin(['miRE.FF5','miRE.FF4'])) & (df_quantiles['group'].isin(['controller','miR'])) & (df_quantiles['design'].isin([0,1])) & (df_quantiles['ts'].isin(['FF6x1','none'])) & (df_quantiles['miR_loc'].isin(['CDS','na']))) | 
                        (df_quantiles['group']=='base'))]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=construct_palette, 
             legend=False, dashes=False, style='construct', markers=construct_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==2), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=construct_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==2)], y='output', hue='construct', palette=construct_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density', ylim=ylim, ylabel='', yticklabels=['']*(len(ax.get_yticklabels())-1))


# stat gmean
ax = axes[2]
plot_df = stats[(stats['promoter']=='EF1a') & 
                       (((stats['miR'].isin(['miRE.FF5','miRE.FF4'])) & (stats['group'].isin(['controller','miR'])) & (stats['design'].isin([0,1])) & (stats['ts'].isin(['FF6x1','none'])) & (stats['miR_loc'].isin(['CDS','na']))) | 
                        (stats['group']=='base'))]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_gmean', hue='construct', palette=construct_palette, 
                    legend=False, ax=ax, marker=construct_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric\nmean', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',
       yticklabels=['']*(len(ax.get_yticklabels())-1),)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata, 'construct', ['miR_', 'ts_'], ax=ax, annotate=False)

# stat std
ax = axes[3]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_std', hue='construct', palette=construct_palette, 
                    legend=False, ax=ax, marker=construct_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard\ndeviation', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',
       yticklabels=['']*(len(ax.get_yticklabels())-1),)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata, 'construct', ['miR_', 'ts_'], ax=ax, annotate=False)

# slope
ax = axes[4]
plot_df = fits[(fits['promoter']=='EF1a') &
                       (((fits['miR'].isin(['miRE.FF5','miRE.FF4'])) & (fits['group'].isin(['controller','miR'])) & (fits['design'].isin([0,1])) & (fits['ts'].isin(['FF6x1','none'])) & (fits['miR_loc'].isin(['CDS','na']))) | 
                        (fits['group']=='base'))]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='slope', hue='construct', palette=construct_palette, 
                    legend=False, ax=ax, marker=construct_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
    
    x_range = np.linspace(np.log10(2.5e2), np.log10(1.2e4), 1000)
    plot_df2 = group[group['biorep']==2]
    axes[0].plot(10**(x_range), 10**(plot_df2['slope'].values[0] * x_range + plot_df2['intercept_log'].values[0]), color='black', zorder=0)

ax.set(title='Slope', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), xlabel='', ylim=(0,1.2), ylabel='',)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata, 'construct', ['miR_', 'ts_'], ax=ax, annotate=False)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist-stat-slope_EF1a-miRE-controls.svg')), bbox_inches='tight')

In [ ]:
# EF1a miR-FF4 vs miRE-FF4 design 1
fig, axes = plt.subplots(1,5, figsize=(15,5), gridspec_kw=dict(width_ratios=(1,0.3,0.4,0.4,0.4), wspace=0.4))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
plot_df = df_quantiles[(((df_quantiles['miR'].isin(['miR.FF4','miRE.FF4'])) & (df_quantiles['group']=='controller') & (df_quantiles['design']==1) & (df_quantiles['ts'].isin(['FF4x1','FF6x1']))) | (df_quantiles['group']=='base')) &
                         (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==2)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=miRE_palette, 
             legend=False, dashes=False, style='construct', markers=miRE_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==2), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=miRE_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==2)], y='output', hue='construct', palette=construct_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density', ylim=ylim, ylabel='', yticklabels=['']*(len(ax.get_yticklabels())-1))


# stat gmean
ax = axes[2]
plot_df = stats[(((stats['miR'].isin(['miR.FF4','miRE.FF4'])) & (stats['group']=='controller') & (stats['design']==1) & (stats['ts'].isin(['FF4x1','FF6x1']))) | (stats['group']=='base')) &
                         (stats['promoter']=='EF1a')]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_gmean', hue='construct', palette=miRE_palette, 
                    legend=False, ax=ax, marker=miRE_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric\nmean', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',
       yticklabels=['']*(len(ax.get_yticklabels())-1), xticklabels=[metadata_dict['ts_kind'][c.get_text()] for c in ax.get_xticklabels()])
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[3]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_std', hue='construct', palette=miRE_palette, 
                    legend=False, ax=ax, marker=miRE_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard\ndeviation', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',
       yticklabels=['']*(len(ax.get_yticklabels())-1), xticklabels=[metadata_dict['ts_kind'][c.get_text()] for c in ax.get_xticklabels()])
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[4]
plot_df = fits[(((fits['miR'].isin(['miR.FF4','miRE.FF4'])) & (fits['group']=='controller') & (fits['design']==1) & (stats['ts'].isin(['FF4x1','FF6x1']))) | (fits['group']=='base')) &
                         (fits['promoter']=='EF1a')]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='slope', hue='construct', palette=miRE_palette, 
                    legend=False, ax=ax, marker=miRE_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
    
    x_range = np.linspace(np.log10(2.5e2), np.log10(1.2e4), 1000)
    plot_df2 = group[group['biorep']==2]
    axes[0].plot(10**(x_range), 10**(plot_df2['slope'].values[0] * x_range + plot_df2['intercept_log'].values[0]), color='black', zorder=0)

ax.set(title='Slope', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), xlabel='', ylim=(0,1.2), ylabel='',
       xticklabels=[metadata_dict['ts_kind'][c.get_text()] for c in ax.get_xticklabels()])
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(output_path/(f'line-marg-hist-stat-slope_EF1a-miRE.FF4.svg')), bbox_inches='tight')